In [1]:
import numpy as np
import os
import ants
from matplotlib import pyplot as plt
import os

from tqdm import tqdm

In [12]:
#fn = ds3542, ds3545, ds1747, ds1796, Ping
fn = 'Ping'
indir = os.path.join('~/Data/Bilingualism_CVAE/output', fn)
indir = os.path.expanduser(indir)
subs = [sub for sub in os.listdir(indir) if sub.startswith('sub-') and not sub.endswith('.html')]
n = len(subs)

outdir = os.path.expanduser(f'~/Documents/Bilingualism_CVAE/data/extracted_brains/{fn}-S{n}')
os.mkdir(outdir)

print(n)

750


In [13]:
def extract_brain(indir,sub,outdir,fn,do_plot=False):    
    anat_fn = '{indir}/{sub}/anat/{sub}_desc-preproc_T1w.nii.gz'.format(indir=indir,sub=sub)
    gm_mask = '{indir}/{sub}/anat/{sub}_label-GM_probseg.nii.gz'.format(indir=indir,sub=sub)
    wm_mask = '{indir}/{sub}/anat/{sub}_label-WM_probseg.nii.gz'.format(indir=indir,sub=sub)
    
# #if fn == 'ds1747'   
#     anat_fn = '{indir}/{sub}/anat/{sub}_run-1_desc-preproc_T1w.nii.gz'.format(indir=indir,sub=sub)
#     gm_mask = '{indir}/{sub}/anat/{sub}_run-1_label-GM_probseg.nii.gz'.format(indir=indir,sub=sub)
#     wm_mask = '{indir}/{sub}/anat/{sub}_run-1_label-WM_probseg.nii.gz'.format(indir=indir,sub=sub)

    t1 = ants.image_read(anat_fn) # filename for raw anatomical 
    gm = ants.image_read(gm_mask) # gray matter probability map
    wm = ants.image_read(wm_mask) # white matter probability map
    brain_mask = gm+wm # combine GM and WM 


    if do_plot:
        t1.plot_ortho(brain_mask,flat=True,xyz_lines=False,orient_labels=False,figsize=5.0,overlay_alpha=.5,overlay_cmap='hot')

    brain  = t1.copy() # Make a copy to not overwrite the original
    
    if do_plot:
        brain_mask.plot_ortho(flat=True,xyz_lines=False,orient_labels=False,figsize=5.0) # Check if it worked

    brain[brain_mask<.9] = 0 # Zero out the non-brain stuff
    if do_plot:
        brain.plot_ortho(flat=True,xyz_lines=False,orient_labels=False,figsize=5.0) # Check if it worked

    if do_plot:
        plt.hist(brain.numpy().flatten()); # check values 

    brain_tiny = brain.resample_image(resample_params=(64,64,64), use_voxels=True, interp_type=4) # Make sure all brains are same size
    
    if do_plot:
        brain_tiny.plot_ortho(flat=True,xyz_lines=False,orient_labels=False,figsize=5.0)

    brain_tiny.to_filename(os.path.join(outdir,sub+'_brain_tiny.nii')) # Save extracted brain
    
    outArr = brain_tiny.numpy()
    outArr = (outArr - np.min(outArr)) / (np.max(outArr) - np.min(outArr))
    
    return outArr

In [14]:
brain_arr = [extract_brain(indir,sub,fn,outdir) for sub in tqdm(subs)]
brain_arr = np.array(brain_arr)

100%|██████████| 750/750 [18:36<00:00,  1.49s/it] 


In [10]:
arr_ofn = os.path.expanduser(f'~/Documents/Bilingualism_CVAE/data/array_brains/Anat-Bilingual-64iso-{fn}-S{n}.npz')
arr_ofn

'/mmfs1/data/liacz/Documents/Bilingualism_CVAE/data/array_brains/Anat-Bilingual-64iso-ds1796-S63.npz'

In [11]:
np.savez_compressed(arr_ofn,data=brain_arr,subs=subs, ds=fn)

In [28]:
# brain_tiny = ants.image_read('~/Documents/Bilingualism_CVAE/data/extracted_brains/ping/sub-P0035_brain_tiny.nii')
# brain_tiny.plot_ortho(flat=True,xyz_lines=False,orient_labels=False,figsize=5.0) 

ValueError: File /mmfs1/data/liacz/Documents/Bilingualism_CVAE/data/extracted_brains/ping/sub-P0035_brain_tiny.nii does not exist!